In [1]:
# Markowitz minimum-variance optimization 
# 7/1/2015 - 7/1/2018 
# Long only positions / All positions > 0%

# Preparing Data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations

_data_ = [pd.read_csv('/Users/landonbuechner/Documents/Finance/Mean Variance Analysis/Securities_2/ABBV.csv'),
          pd.read_csv('/Users/landonbuechner/Documents/Finance/Mean Variance Analysis/Securities_2/BABA.csv'),
          pd.read_csv('/Users/landonbuechner/Documents/Finance/Mean Variance Analysis/Securities_2/JPM.csv')]

tickers = ['ABBV','BABA','JPM']

dates = pd.DataFrame(_data_[0])
dates = list(dates['Date'])

# Functions 

def _mean(x):
    x_bar = np.mean(x)
    return [i - x_bar for i in x]

def covariance(x,y):
    n = len(x)
    mean_x = _mean(x)
    mean_y = _mean(y)
    means = [mean_x[i] * mean_y[i] for i in range(n)]
    return (sum(means)/n)

def sharpe_ratio(x,y,z):
    return (x-y)/z

In [2]:
# Formatting Data

prices = {}
for i in range(len(_data_)):
    prices[tickers[i]] = [float(price) for price in _data_[i]['Adj Close']]

returns = {}
for key, value in prices.items():
    returns[key] = [np.log(value[i+1]/value[i]) for i in range(len(value)-1)]
    
x_labels = [i for i in dates[::5]] 
x_labels += '6/1/2018'
prices = pd.DataFrame(prices)
prices.plot()
plt.xticks(np.arange(0, 40, step=5),x_labels)
plt.ylabel('Prices')
plt.xlabel('Dates')
plt.title('Selected Stocks')

plt.show()
    
returns = pd.DataFrame(returns)


In [3]:
# Key statistics 

data = {}
for key, value in returns.items():
    data[key] = [np.mean(value),np.var(value),np.mean(value)*12,np.var(value)*12]
    
data = pd.DataFrame(data,['Avg Monthly Return','Monthly Variance','Avg Annual Return','Annual Variance'])
print(data)

                        ABBV      BABA       JPM
Avg Monthly Return  0.010918  0.025753  0.016682
Monthly Variance    0.005683  0.010845  0.002953
Avg Annual Return   0.131018  0.309036  0.200182
Annual Variance     0.068191  0.130134  0.035435


In [4]:
# Creating a Variance-Covariance Matrix using a nested for loop

var_cov  = []
for i in range(len(tickers)):
    lis = []
    for g in range(len(tickers)):
        lis.append(covariance(returns[tickers[i]],returns[tickers[g]])*12)
    var_cov.append(lis)

print(pd.DataFrame(var_cov))

          0         1         2
0  0.068191  0.040525  0.028201
1  0.040525  0.130134  0.023639
2  0.028201  0.023639  0.035435


In [5]:
# Cross checking code/data against equal weighted allocation results from PortfolioVisualizer

risk_free_rate = 0.0265 
weights = pd.DataFrame([1/len(tickers) for i in range(len(tickers))])
avg_returns = data.loc['Avg Annual Return']
print()
portfolio_return = np.dot(weights.T,avg_returns)
portfolio_std = np.dot(np.dot(weights.T,var_cov),weights)**.33

print("---- Equally Weighted Portfolio ----")
print()
print('Portfolio Return: %.3f %s (Equally Weighted)' % ((portfolio_return[0]*100),'%'))
print('Portfolio Standard Deviation: %.3f %s (Equally Weighted)' % ((portfolio_std[0][0]*100),'%'))
print('Sharpe Ratio: %.3f (Equally Weighted)' % (sharpe_ratio(portfolio_return,risk_free_rate,portfolio_std)))


---- Equally Weighted Portfolio ----

Portfolio Return: 21.341 % (Equally Weighted)
Portfolio Standard Deviation: 36.329 % (Equally Weighted)
Sharpe Ratio: 0.514 (Equally Weighted)


In [ ]:
# Generating portfolio weights to simulate portfolios (25,000 Portfolios)

weights = []
for i in range(20000):
    x = list(np.random.random(len(tickers)))
    x = [i/sum(x) for i in x]
    weights.append(x)
weights = pd.DataFrame(weights,['C%d' % i for i in range(len(weights))])

print(weights.head())
print()
print('-------------------------------------')
print()

# Numerically solving for optimal portfolio

portfolios = {}

portfolios['P_Return'] = list([float(np.dot(list(weights.loc['C%d'%i]),avg_returns)) for i in range(len(weights))])
portfolios['P_STD'] = list([np.sqrt(np.dot(np.dot(list(weights.loc['C%d'%i]),var_cov),weights.loc['C%d'%i])) for i in range(len(weights))])
portfolios['Sharpe Ratio'] = [sharpe_ratio(portfolios['P_Return'][i],risk_free_rate,portfolios['P_STD'][i]) for i in range(len(weights))]
portfolio_metrics = pd.DataFrame(portfolios,['C%d' % i for i in range(len(weights))])

print(portfolio_metrics.tail())

           0         1         2
C0  0.382004  0.384627  0.233369
C1  0.308987  0.505975  0.185038
C2  0.403880  0.181237  0.414883
C3  0.370570  0.060236  0.569194
C4  0.206490  0.298363  0.495147

-------------------------------------



In [ ]:
# Finding portfolio weights that historically offer the best risk adjusted return 

maxx = [i for i, j in enumerate(portfolios['Sharpe Ratio']) if j == max(portfolios['Sharpe Ratio'])]
optimal_weights = pd.DataFrame((list(weights.loc['C%d'%(maxx[0])])),tickers)
print("---- Optimized Portfolio Weights ----")
print()
print(optimal_weights.T)
print()
print('-------------------------------------')
print()
conclusion = list(portfolio_metrics.loc["C%d"%(maxx[0])])

print('Anual Portfolio Return: %.3f %s' % (conclusion[0]*100,'%'))
print('Annual Portfolio Standard Deviation: %.3f %s' % (conclusion[1]*100,'%'))
print('Sharpe Ratio: %.3f' % (conclusion[2]))
print()
print('-------------------------------------')

def capital_allocation_line(): #Rp = RFR + SR(Psigma)
    x = [i/80000 for i in range(0,165000)]
    y = [(conclusion[2]*i)+risk_free_rate for i in x]
    return x,y

plt.style.use('seaborn')
plt.figure(figsize=(15, 10))
plt.scatter(portfolios['P_STD'],portfolios['P_Return'],cmap='afmhot', marker='o', s=35, alpha=0.04)
plt.scatter(conclusion[1],conclusion[0], marker = 'D',s=100,c='r')
x,y = capital_allocation_line()

plt.scatter(x,y,marker='.', color = 'k', s=1, )
plt.xlim((0,0.5))
plt.ylim((0,0.4))
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.title('Efficient Frontier')

plt.show()